In [127]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os

In [184]:
#Словарь с неисправностей
fault_type_list = {
    1:"Нормальная работа",
    2:"Влияние газа",
    3:"Утечки в нагнетательной части",
    4:"Утечки в приемной части",
    5:"Обрыв/отворот",
    6:"АСПО",
    7:"Высокая посадка плунжера",
    8:"Низкая посадка плунжера",
    9:"Выход плунжера из цилиндра",
    10:"Прихват плунжера",
    11:"Заедание плунжера"}

In [128]:
# Предобработка данных
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),  # Изменение размера до 256x256 пикселей
        transforms.ToTensor(),  # Преобразование в тензор (многомерный массив)
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),  # Изменение размера до 256x256 пикселей
        transforms.ToTensor(),  # Преобразование в тензор
    ]),
}


In [129]:
data_dir = 'img/dyn'  # Каталог с изображениями
batch_size = 32  # Размер пакета данных для обучения

In [168]:
# Создание ImageFolder датасета для обучения и валидации с использованием заданных трансформаций
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 325
     Root location: img/dyn/train
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
                ToTensor()
            ),
 'test': Dataset ImageFolder
     Number of datapoints: 73
     Root location: img/dyn/test
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
                ToTensor()
            )}

In [131]:
# Создание DataLoader для загрузки данных с пакетами, перемешиванием и указанием числа рабочих процессов
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=0) for x in ['train', 'test']}

In [132]:
# Определение размеров датасетов для обучения и валидации
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
dataset_sizes

{'train': 325, 'test': 73}

In [133]:
# Получение списка классов
class_names = image_datasets['train'].classes
class_names

['1', '10', '2', '3', '4', '5', '6', '7', '8', '9']

In [134]:
# Загрузка предварительно обученной модели ResNet18
model = models.resnet18(pretrained=True)

In [135]:
# Получение количества признаков в последнем полносвязном слое
num_ftrs = model.fc.in_features
num_ftrs

512

In [136]:
# Замена последнего полносвязного слоя на слой с 10 выходами
model.fc = nn.Linear(num_ftrs, 10)
model.fc

Linear(in_features=512, out_features=10, bias=True)

In [137]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # Определение устройства (GPU или CPU)
device

device(type='cuda', index=0)

In [138]:
model = model.to(device)  # Перемещение модели на выбранное устройство

In [139]:
# Определение функции потерь (cross-entropy) и оптимизатора (SGD)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [140]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() # стохастический градиентный спуск для обновления весов модели
        outputs = model(inputs)
        loss = criterion(outputs, labels) # принимает предсказанные значения модели и истинные (ожидаемые) значения (метки) и вычисляет, насколько они различаются.
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / dataset_sizes['train']}")

Epoch 1, Loss: 0.0728218203324538
Epoch 2, Loss: 0.04980806864224947
Epoch 3, Loss: 0.038479567491091216
Epoch 4, Loss: 0.027907684766329252
Epoch 5, Loss: 0.02291605839362511
Epoch 6, Loss: 0.018677076743199274
Epoch 7, Loss: 0.014109523433905381
Epoch 8, Loss: 0.010975884611789997
Epoch 9, Loss: 0.007991544971099267
Epoch 10, Loss: 0.007258408849055951


In [141]:
# Сохранение модели
torch.save(model.state_dict(), 'dyn_classification_model.pth')

In [142]:
# Классификация нового изображения
from PIL import Image
from torchvision import transforms

def classify_image(image_path):
    image = Image.open(image_path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ])
    image = preprocess(image).unsqueeze(0)  # Добавление размерности батча (batch dimension)
    image = image.to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    return predicted[0]

In [187]:
#Демонстрационная выборка
for i in range(1, 10):
    new_image_path = f'img/demo/{i}.jpg'
    predicted_class = classify_image(new_image_path)
    print(f'{i}=: {predicted_class}')

1=: 6
2=: 4
3=: 3
4=: 0
5=: 5
6=: 6
7=: 7
8=: 9
9=: 9


In [185]:
tes_dir = 'img/dyn/test'
tpoz = 0
tneg = 0
for i in range(1, 11):
    files_path = f'img/dyn/test/{i}'
    files = os.listdir(files_path)
    poz = 0
    neg = 0
    for file_name in files:
        full_file_name = f'{files_path}/{file_name}'
        predicted_class = classify_image(full_file_name)
        if predicted_class == i:
            poz += 1
        else:
            neg += 1
    tpoz += poz
    tneg += neg
    print(f'{fault_type_list[i]}:+{poz}-{neg}={poz / (poz + neg) }')
print(f'total:+{tpoz}-{tneg}={tpoz / (tpoz + tneg) }')

Нормальная работа:+0-10=0.0
Влияние газа:+7-0=1.0
Утечки в нагнетательной части:+4-1=0.8
Утечки в приемной части:+4-1=0.8
Обрыв/отворот:+11-1=0.9166666666666666
АСПО:+10-0=1.0
Высокая посадка плунжера:+6-1=0.8571428571428571
Низкая посадка плунжера:+8-0=1.0
Выход плунжера из цилиндра:+8-1=0.8888888888888888
Прихват плунжера:+0-1=0.0
total:+58-16=0.7837837837837838
